Define the brickcomps 

In [167]:
using DataFrames
using CSV
AIS_Data = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP85/projections_antarctic_RCP85_sneasybrick.csv") |> DataFrame;

In [159]:
GMSL_Data = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP85/projections_gmsl_RCP85_sneasybrick.csv") |> DataFrame;

In [160]:
LWS_Data = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP85/projections_landwater_storage_sl_RCP85_sneasybrick.csv") |> DataFrame;

In [161]:
TE_Data = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP85/projections_thermal_RCP85_sneasybrick.csv") |> DataFrame;

In [163]:
GIS_Data = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP85/projections_greenland_RCP85_sneasybrick.csv") |> DataFrame;

In [164]:
GSIC_Data = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP85/projections_glaciers_RCP85_sneasybrick.csv") |> DataFrame;

Construct Brick components

In [124]:
Sim=10000
GMSL=GMSL_Data[:,1:Sim];
LWS=LWS_Data[:,1:Sim];
GIS= GIS_Data[:,1:Sim];
GSIC=GSIC_Data[:,1:Sim];
TE=TE_Data[:,1:Sim];
AIS=AIS_Data[:,1:Sim];

In [113]:
Y=collect(1850:2300);

btime=Y;



In [125]:
brickcomps=Tuple([btime,AIS,GSIC,GIS,TE,LWS,GMSL]);

Run Brick downscale function

In [115]:
function get_fingerprints()

    fp_dir = joinpath(@__DIR__, "..", "data")
    isdir(fp_dir) || mkpath(fp_dir)
    fp_file = joinpath(fp_dir, "FINGERPRINTS_SLANGEN_Bakker.nc")
    if !isfile(fp_file)
        url = "https://github.com/scrim-network/BRICK/raw/master/fingerprints/FINGERPRINTS_SLANGEN_Bakker.nc"
        download(url, fp_file)
    end

    fplat = ncread(fp_file,"lat")
    fplon = ncread(fp_file,"lon")
    fpAIS = ncread(fp_file,"AIS")
    fpGSIC = ncread(fp_file,"GLAC")
    fpGIS = ncread(fp_file,"GIS")
    ncclose()

    return fplat,fplon,fpAIS,fpGSIC,fpGIS
end

get_fingerprints (generic function with 1 method)

In [116]:
function subtractor(minval,maxval)
    function y(point,n)
        if point - n < minval
            return min(maxval,point - n + maxval)
        else
            return point - n
        end
    end
end

subtractor (generic function with 1 method)

In [117]:
function adder(maxval)
    function y(point,n)
        if point + n > maxval
            return point + n - maxval
        else
            return point + n
        end
    end
end

adder (generic function with 1 method)

In [118]:
lon_subtractor = subtractor(1,360)
lon_adder = adder(360)
lat_adder = adder(180)
lat_subtractor = subtractor(1,180)

(::var"#y#209"{Int64, Int64}) (generic function with 1 method)

In [119]:
using Statistics
using NetCDF
function downscale_brick(brickcomps, lonlat, ensInds, ystart=2010, yend=2100, tstep=10)
    # To do - check with vectors of lat, lon
    (fplat,fplon,fpAIS,fpGSIC,fpGIS) = get_fingerprints()
    (btime,AIS,GSIC,GIS,TE,LWS,GMSL) = brickcomps

    # Select indices of time of interest, with respect to timestep
    tinds = findall(x -> x .>= ystart && x .<=yend, btime)
    years = collect(ystart:yend)
    yinds = findall(x -> x % tstep==0, years)
    # Need to normalize LSL relative to 2000
    inorm = findall(x -> x==2000, btime)

    tdim=length(btime)

    if length(years)==length(tinds)
        tinds = tinds[yinds]
    else
        println("Error: years outside of bounds")
        return nothing
    end

    num_ens = length(ensInds)
    
    # Output matrix: ens x time x segment
    lsl_out = zeros(num_ens, length(tinds), length(lonlat))

    # Trim component vectors to timesteps and ensembles. Assume interval is 1 year
    if tdim==size(AIS)[1] # check that time dimension is 1
        # for normalizing
        AIS_norm = AIS[inorm,ensInds]
        GSIC_norm = GSIC[inorm,ensInds]
        GIS_norm = GIS[inorm,ensInds]
        TE_norm = TE[inorm,ensInds]
        LWS_norm = LWS[inorm,ensInds]
        GMSL_norm = GMSL[inorm,ensInds]
        # actual projections
        AIS = AIS[tinds,ensInds]
        GSIC = GSIC[tinds,ensInds]
        GIS = GIS[tinds,ensInds]
        TE = TE[tinds,ensInds]
        LWS = LWS[tinds,ensInds]
        GMSL = GMSL[tinds,ensInds]
    else
        println("Error: time dimension is not 1 for brick components")
        return nothing
    end

  

    for f in 1:length(lonlat) # Loop through lonlat tuples

        lon = lonlat[f][1]
        lat = lonlat[f][2]
        # Convert Longitude to degrees East
        # CIAM Lat is already in (-90,90) by default
        if lon <0
            lon = lon + 360
        end

        # Find fingerprint degrees nearest to lat,lon
        ilat = findall(isequal(minimum(abs.(fplat.-lat))),abs.(fplat.-lat))
        ilon = findall(isequal(minimum(abs.(fplon.-lon))),abs.(fplon.-lon))


        # Take average of closest lat/lon values
        fpAIS_flat = collect(skipmissing(Iterators.flatten(fpAIS[ilon,ilat])))
        fpGSIC_flat = collect(skipmissing(Iterators.flatten(fpGSIC[ilon,ilat])))
        fpGIS_flat = collect(skipmissing(Iterators.flatten(fpGIS[ilon,ilat])))

        fpAIS_loc = mean(fpAIS_flat[isnan.(fpAIS_flat).==false],dims=1)[1]
        fpGSIC_loc = mean(fpGSIC_flat[isnan.(fpGSIC_flat).==false],dims=1)[1]
        fpGIS_loc = mean(fpGIS_flat[isnan.(fpGIS_flat).==false],dims=1)[1]
        fpTE_loc = 1.0
        fpLWS_loc=1.0

        # Keep searching nearby lat/lon values if fingerprint value is NaN unless limit is hit
        inc =1

        while isnan(fpAIS_loc) || isnan(fpGIS_loc) || isnan(fpGSIC_loc) && inc<5

            newlonStart = lon_subtractor.(fplon[ilon],inc)[1]
            newlatStart = lat_subtractor.(fplat[ilat],inc)[1]
            newlonEnd = lon_adder.(fplon[ilon],inc)[1]
            newlatEnd = lat_adder.(fplat[ilat],inc)[1]

            latInd1 = minimum(findall(isequal(minimum(abs.(fplat.-newlatStart))),abs.(fplat.-newlatStart)))
            #minimum(findall(x-> x in newlatStart,fplat))
            latInd2 = maximum(findall(isequal(minimum(abs.(fplat.-newlatEnd))),abs.(fplat.-newlatEnd)))
            #maximum(findall(x -> x in newlatEnd,fplat))

            lonInd1 = minimum(findall(isequal(minimum(abs.(fplon.-newlonStart))),abs.(fplon.-newlonStart)))
            #minimum(findall(x-> x in newlonStart,fplon))
            lonInd2 = maximum(findall(isequal(minimum(abs.(fplon.-newlonEnd))),abs.(fplon.-newlonEnd)))
            #maximum(findall(x -> x in newlonEnd,fplon))

            if latInd2 < latInd1
                latInds=[latInd1; 1:latInd2]
            else
                latInds=latInd1:latInd2
            end

            if lonInd2 < lonInd1
                lonInds=[lonInd1; 1:lonInd2]
            else
                lonInds = lonInd1:lonInd2
            end

            fpAIS_flat = collect(skipmissing(Iterators.flatten(fpAIS[lonInds,latInds])))
            fpGSIC_flat = collect(skipmissing(Iterators.flatten(fpGSIC[lonInds,latInds])))
            fpGIS_flat = collect(skipmissing(Iterators.flatten(fpGIS[lonInds,latInds])))

            fpAIS_loc = mean(fpAIS_flat[isnan.(fpAIS_flat).==false],dims=1)[1]
            fpGSIC_loc = mean(fpGSIC_flat[isnan.(fpGSIC_flat).==false],dims=1)[1]
            fpGIS_loc = mean(fpGIS_flat[isnan.(fpGIS_flat).==false],dims=1)[1]

            inc = inc + 1

        end

        # If still NaN, throw an error
        if isnan(fpAIS_loc) || isnan(fpGIS_loc) || isnan(fpGSIC_loc)
            println("Error: no fingerprints found for ($(lon),$(lat))")
            return nothing
        end
       # Multiply fingerprints by BRICK ensemble members
       if ndims(AIS) > 1
        for n in 1:size(AIS)[2] # loop through ensemble members
            lsl_out[n, :, f] = fpGIS_loc * GIS[:,n] + fpAIS_loc * AIS[:,n] + fpGSIC_loc * GSIC[:,n] +
                               fpTE_loc * TE[:,n] + fpLWS_loc * LWS[:,n]
            # CIAM - LSL should be sea-level change relative to year 2000
            lsl_norm = fpGIS_loc * GIS_norm[1,n] + fpAIS_loc * AIS_norm[1,n] + fpGSIC_loc * GSIC_norm[1,n] +
                       fpTE_loc * TE_norm[1,n] + fpLWS_loc * LWS_norm[1,n]
            lsl_out[n, :, f] = lsl_out[n, :, f] .- lsl_norm
            
        end
    else
            lsl_out[1, :, f] = fpGIS_loc * GIS[:] + fpAIS_loc * AIS[:] + fpGSIC_loc * GSIC[:] +
                                fpTE_loc * TE[:] + fpLWS_loc * LWS[:]
            # CIAM - LSL should be sea-level change relative to year 2000
            lsl_norm = fpGIS_loc * GIS_norm + fpAIS_loc * AIS_norm + fpGSIC_loc * GSIC_norm +
                                fpTE_loc * TE_norm + fpLWS_loc * LWS_norm
            lsl_out[1, :, f] = lsl_out[1, :, f] .- lsl_norm
    end

    end # End lonlat tuple

    return lsl_out,GMSL
end

downscale_brick (generic function with 4 methods)

In [139]:
ensInds=collect(1:10000);
lonlat= ([-104.3385,19.1138],[-90.0715,29.9511]);
ystart=2010;
yend=2200;
tstep=10;

lslrNew,H=downscale_brick(brickcomps, lonlat, ensInds, ystart, yend, tstep);

([0.02676421779527241 0.06812314170115447 … 6.361669572580161 6.862617183909249; 0.024081657354711766 0.06299086519819136 … 3.673713132631384 3.99032120250646; … ; 0.032181918599016354 0.07211076206309919 … 5.156145274576946 5.615677439153576; 0.030560387249516846 0.06991447462590339 … 6.44974410335111 6.9719984662454415]

[0.025283754282903553 0.06390356159746934 … 5.682386744769553 6.1214555837968625; 0.02304732387304978 0.059257821539559385 … 3.3212798100616925 3.60458242479375; … ; 0.030655010921650183 0.06810956394438224 … 4.535057511755059 4.9348544856717895; 0.029054709859027222 0.06582189420482644 … 5.6864725459669945 6.140884458605818], 20×10000 DataFrame
 Row │ x1         x2         x3         x4         x5         x6         x7    ⋯
     │ Float64    Float64    Float64    Float64    Float64    Float64    Float ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ 0.0835504  0.0814309  0.0773876  0.0827359  0.0704417  0.0788344  0.066 ⋯
   

Save the Local SLR for Manzanillo, Colima and New Orleans in a CVS File 
1) Make a data frame for each one
2) Save it as a CVS File

RCP 8.5 case

In [150]:
#Manzanillo, Colima, Mexico Matrix
COLlslr85=zeros(20,10001);
#First colum are the years 2010 to 2200 with time step of 10 years
for i in 1:20
    COLlslr85[i,1]=2000+i*10
end
#SLR value for each year for each simulation 
for i in 1:10000
    for j in 1:20
        COLlslr85[j,i+1]=lslrNew[i,j,1]
    end 
end 

In [220]:
#New Orleans Matrix
NOlslr85=zeros(20,10001);
#First colum are the years 2010 to 2200 with time step of 10 years
for i in 1:20
    NOlslr85[i,1]=2000+i*10
end
#SLR value for each year for each simulation 
for i in 1:10000
    for j in 1:20
        NOlslr85[j,i+1]=lslrNew[i,j,2]
    end 
end 

In [207]:
Head=zeros(1,10001);
Head[1,1]="Years"

MethodError: MethodError: Cannot `convert` an object of type String to an object of type Float64
Closest candidates are:
  convert(::Type{T}, !Matched::DoubleFloats.DoubleFloat{DoubleFloats.DoubleFloat{T}}) where T<:Union{Float16, Float32, Float64} at /Users/ce3304/.julia/packages/DoubleFloats/6UuOC/src/type/convert.jl:37
  convert(::Type{T}, !Matched::Mimi.ScalarModelParameter{T}) where T at /Users/ce3304/.julia/packages/Mimi/OClro/src/core/types/params.jl:46
  convert(::Type{T}, !Matched::DualNumbers.Dual) where T<:Union{Real, Complex} at /Users/ce3304/.julia/packages/DualNumbers/5knFX/src/dual.jl:24
  ...

In [147]:
import Pkg 
Pkg.add("Tables")


In [221]:
using CSV
using Tables
CSV.write("projections_Col_LSLR_RCP85.csv", Tables.table(COLlslr85))
CSV.write("projections_NO_LSLR_RCP85.csv", Tables.table(NOlslr85))

"projections_NO_LSLR_RCP85.csv"

RCP 6.0 case

In [166]:
using DataFrames
using CSV
AIS_Data60 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP60/projections_antarctic_RCP60_sneasybrick.csv") |> DataFrame;

In [178]:
GMSL_Data60 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP60/projections_gmsl_RCP60_sneasybrick.csv") |> DataFrame;

In [179]:
LWS_Data60 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP60/projections_landwater_storage_sl_RCP60_sneasybrick.csv") |> DataFrame;

In [176]:
TE_Data60 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP60/projections_thermal_RCP60_sneasybrick.csv") |> DataFrame;

In [183]:
GIS_Data60 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP60/projections_greenland_RCP60_sneasybrick.csv") |> DataFrame;

In [186]:
GSIC_Data60 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP60/projections_glaciers_RCP60_sneasybrick.csv") |> DataFrame;

In [189]:
Sim=10000
GMSL60=GMSL_Data60[:,1:Sim];
LWS60=LWS_Data60[:,1:Sim];
GIS60= GIS_Data60[:,1:Sim];
GSIC60=GSIC_Data60[:,1:Sim];
TE60=TE_Data60[:,1:Sim];
AIS60=AIS_Data60[:,1:Sim];

In [192]:
brickcomps60=Tuple([btime,AIS60,GSIC60,GIS60,TE60,LWS60,GMSL60]);

In [195]:
ensInds=collect(1:10000);
lonlat= ([-104.3385,19.1138],[-90.0715,29.9511]);
ystart=2010;
yend=2200;
tstep=10;

lslrNew60,H60=downscale_brick(brickcomps60, lonlat, ensInds, ystart, yend, tstep);

In [198]:
#Manzanillo, Colima, Mexico Matrix
COLlslr60=zeros(20,10001);
#First colum are the years 2010 to 2200 with time step of 10 years
for i in 1:20
    COLlslr60[i,1]=2000+i*10
end
#SLR value for each year for each simulation 
for i in 1:10000
    for j in 1:20
        COLlslr60[j,i+1]=lslrNew60[i,j,1]
    end 
end 

In [218]:
#New Orleans Matrix
NOlslr60=zeros(20,10001);
#First colum are the years 2010 to 2200 with time step of 10 years
for i in 1:20
    NOlslr60[i,1]=2000+i*10
end
#SLR value for each year for each simulation 
for i in 1:10000
    for j in 1:20
        NOlslr60[j,i+1]=lslrNew60[i,j,2]
    end 
end 

In [219]:
using CSV
using Tables
CSV.write("projections_Col_LSLR_RCP60.csv", Tables.table(COLlslr60))
CSV.write("projections_NO_LSLR_RCP60.csv", Tables.table(NOlslr60))

"projections_NO_LSLR_RCP60.csv"

RCP 4.5 case

In [171]:
AIS_Data45 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP45/projections_antarctic_RCP45_sneasybrick.csv") |> DataFrame;

In [177]:
GMSL_Data45= CSV.File("/Users/ce3304/Downloads/projections_csv/RCP45/projections_gmsl_RCP45_sneasybrick.csv") |> DataFrame;

In [169]:
LWS_Data45 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP45/projections_landwater_storage_sl_RCP45_sneasybrick.csv") |> DataFrame;

In [180]:
TE_Data45 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP45/projections_thermal_RCP45_sneasybrick.csv") |> DataFrame;

In [184]:
GIS_Data45 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP45/projections_greenland_RCP45_sneasybrick.csv") |> DataFrame;

In [187]:
GSIC_Data45 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP45/projections_glaciers_RCP45_sneasybrick.csv") |> DataFrame;

In [190]:
Sim=10000
GMSL45=GMSL_Data45[:,1:Sim];
LWS45=LWS_Data45[:,1:Sim];
GIS45= GIS_Data45[:,1:Sim];
GSIC45=GSIC_Data45[:,1:Sim];
TE45=TE_Data45[:,1:Sim];
AIS45=AIS_Data45[:,1:Sim];

In [193]:
brickcomps45=Tuple([btime,AIS45,GSIC45,GIS45,TE45,LWS45,GMSL45]);

In [196]:
ensInds=collect(1:10000);
lonlat= ([-104.3385,19.1138],[-90.0715,29.9511]);
ystart=2010;
yend=2200;
tstep=10;

lslrNew45,H=downscale_brick(brickcomps45, lonlat, ensInds, ystart, yend, tstep);

In [199]:
#Manzanillo, Colima, Mexico Matrix
COLlslr45=zeros(20,10001);
#First colum are the years 2010 to 2200 with time step of 10 years
for i in 1:20
    COLlslr45[i,1]=2000+i*10
end
#SLR value for each year for each simulation 
for i in 1:10000
    for j in 1:20
        COLlslr45[j,i+1]=lslrNew45[i,j,1]
    end 
end 

In [216]:
#New Orleans Matrix
NOlslr45=zeros(20,10001);
#First colum are the years 2010 to 2200 with time step of 10 years
for i in 1:20
    NOlslr45[i,1]=2000+i*10
end
#SLR value for each year for each simulation 
for i in 1:10000
    for j in 1:20
        NOlslr45[j,i+1]=lslrNew45[i,j,2]
    end 
end 

In [217]:
using CSV
using Tables
CSV.write("projections_Col_LSLR_RCP45.csv", Tables.table(COLlslr45))
CSV.write("projections_NO_LSLR_RCP45.csv", Tables.table(NOlslr45))

"projections_NO_LSLR_RCP45.csv"

RCP 2.6 case

In [181]:
AIS_Data26 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP26/projections_antarctic_RCP26_sneasybrick.csv") |> DataFrame;

In [174]:
GMSL_Data26 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP26/projections_gmsl_RCP26_sneasybrick.csv") |> DataFrame;

In [170]:
LWS_Data26 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP26/projections_landwater_storage_sl_RCP26_sneasybrick.csv") |> DataFrame;

In [182]:
TE_Data26 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP26/projections_thermal_RCP26_sneasybrick.csv") |> DataFrame;

In [185]:
GIS_Data26 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP26/projections_greenland_RCP26_sneasybrick.csv") |> DataFrame;

In [188]:
GSIC_Data26 = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP26/projections_glaciers_RCP26_sneasybrick.csv") |> DataFrame;

In [191]:
Sim=10000
GMSL26=GMSL_Data26[:,1:Sim];
LWS26=LWS_Data26[:,1:Sim];
GIS26= GIS_Data26[:,1:Sim];
GSIC26=GSIC_Data26[:,1:Sim];
TE26=TE_Data26[:,1:Sim];
AIS26=AIS_Data26[:,1:Sim];

In [194]:
brickcomps26=Tuple([btime,AIS26,GSIC26,GIS26,TE26,LWS26,GMSL26]);

In [197]:
ensInds=collect(1:10000);
lonlat= ([-104.3385,19.1138],[-90.0715,29.9511]);
ystart=2010;
yend=2200;
tstep=10;

lslrNew26,H=downscale_brick(brickcomps26, lonlat, ensInds, ystart, yend, tstep);

In [200]:
#Manzanillo, Colima, Mexico Matrix
COLlslr26=zeros(20,10001);
#First colum are the years 2010 to 2200 with time step of 10 years
for i in 1:20
    COLlslr26[i,1]=2000+i*10
end
#SLR value for each year for each simulation 
for i in 1:10000
    for j in 1:20
        COLlslr26[j,i+1]=lslrNew26[i,j,1]
    end 
end 

In [213]:
#New Orleans Matrix
NOlslr26=zeros(20,10001);
#First colum are the years 2010 to 2200 with time step of 10 years
for i in 1:20
    NOlslr26[i,1]=2000+i*10
end
#SLR value for each year for each simulation 
for i in 1:10000
    for j in 1:20
        NOlslr26[j,i+1]=lslrNew26[i,j,2]
    end 
end 

In [215]:
using CSV
using Tables
CSV.write("projections_Col_LSLR_RCP26.csv", Tables.table(COLlslr26))
CSV.write("projections_NO_LSLR_RCP26.csv", Tables.table(NOlslr26))

"projections_NO_LSLR_RCP26.csv"

In [128]:
using MimiCIAM
using Mimi
M=zeros(1,10);
for i in 1:10
    new_lslr=zeros(20,2);
    for j in 1:2
        for k in 1:20
        new_lslr[k,j]=lslrNew[i,k,j];
        end 
     end 
    m = MimiCIAM.get_model(initfile ="../ColimaUSA3_init.csv")
    update_param!(m, :slrcost, :lslr, new_lslr)
    M[i]=run(m)
end

MethodError: MethodError: Cannot `convert` an object of type Nothing to an object of type Float64
Closest candidates are:
  convert(::Type{T}, !Matched::DoubleFloats.DoubleFloat{DoubleFloats.DoubleFloat{T}}) where T<:Union{Float16, Float32, Float64} at /Users/ce3304/.julia/packages/DoubleFloats/6UuOC/src/type/convert.jl:37
  convert(::Type{T}, !Matched::Mimi.ScalarModelParameter{T}) where T at /Users/ce3304/.julia/packages/Mimi/OClro/src/core/types/params.jl:46
  convert(::Type{T}, !Matched::DualNumbers.Dual) where T<:Union{Real, Complex} at /Users/ce3304/.julia/packages/DualNumbers/5knFX/src/dual.jl:24
  ...